In [1]:
! git clone https://github.com/sirius-ai/LPRNet_Pytorch

Cloning into 'LPRNet_Pytorch'...
remote: Enumerating objects: 1071, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1071 (delta 25), reused 22 (delta 22), pack-reused 1037 (from 1)
Receiving objects: 100% (1071/1071), 20.04 MiB | 8.16 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [2]:
 cd LPRNet_Pytorch/

/content/LPRNet_Pytorch


In [14]:
# Base Model
! python test_LPRNet.py

Successful to build network!
/content/LPRNet_Pytorch/test_LPRNet.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.load_state_dict(torch.load(args.pretrained_model

In [15]:
# Normal Pruning
! python test_LPRNet.py --pretrained_model ./weights/pruned_model_weights_trial2.pth

Model: ./weights/pruned_model_weights_trial2.pth
Successful to build network!
/content/LPRNet_Pytorch/test_LPRNet.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet

In [16]:
# Pro Pruning
! python test_LPRNet.py --pretrained_model ./weights/pruned_model_weights_trial.pth

Model: ./weights/pruned_model_weights_trial.pth
Successful to build network!
/content/LPRNet_Pytorch/test_LPRNet.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lprnet.

In [3]:
import torch.nn as nn
import torch

class small_basic_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(small_basic_block, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(ch_in, ch_out // 4, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out // 4, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.Conv2d(ch_out // 4, ch_out, kernel_size=1),
        )
    def forward(self, x):
        return self.block(x)

class LPRNet(nn.Module):
    def __init__(self, lpr_max_len, phase, class_num, dropout_rate):
        super(LPRNet, self).__init__()
        self.phase = phase
        self.lpr_max_len = lpr_max_len
        self.class_num = class_num
        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1), # 0
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),  # 2
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1)),
            small_basic_block(ch_in=64, ch_out=128),    # *** 4 ***
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),  # 6
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2)),
            small_basic_block(ch_in=64, ch_out=256),   # 8
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),  # 10
            small_basic_block(ch_in=256, ch_out=256),   # *** 11 ***
            nn.BatchNorm2d(num_features=256),   # 12
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(4, 1, 2)),  # 14
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=(1, 4), stride=1),  # 16
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),  # 18
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=256, out_channels=class_num, kernel_size=(13, 1), stride=1), # 20
            nn.BatchNorm2d(num_features=class_num),
            nn.ReLU(),  # *** 22 ***
        )
        self.container = nn.Sequential(
            nn.Conv2d(in_channels=448+self.class_num, out_channels=self.class_num, kernel_size=(1, 1), stride=(1, 1)),
            # nn.BatchNorm2d(num_features=self.class_num),
            # nn.ReLU(),
            # nn.Conv2d(in_channels=self.class_num, out_channels=self.lpr_max_len+1, kernel_size=3, stride=2),
            # nn.ReLU(),
        )

    def forward(self, x):
        keep_features = list()
        for i, layer in enumerate(self.backbone.children()):
            x = layer(x)
            if i in [2, 6, 13, 22]: # [2, 4, 8, 11, 22]
                keep_features.append(x)

        global_context = list()
        for i, f in enumerate(keep_features):
            if i in [0, 1]:
                f = nn.AvgPool2d(kernel_size=5, stride=5)(f)
            if i in [2]:
                f = nn.AvgPool2d(kernel_size=(4, 10), stride=(4, 2))(f)
            f_pow = torch.pow(f, 2)
            f_mean = torch.mean(f_pow)
            f = torch.div(f, f_mean)
            global_context.append(f)

        x = torch.cat(global_context, 1)
        x = self.container(x)
        logits = torch.mean(x, dim=2)

        return logits

def build_lprnet(lpr_max_len=8, phase=False, class_num=66, dropout_rate=0.5):

    Net = LPRNet(lpr_max_len, phase, class_num, dropout_rate)

    if phase == "train":
        return Net.train()
    else:
        return Net.eval()

In [4]:
def load_model(weight_path, lpr_max_len=7, phase='test', class_num=68, dropout_rate=0.5):

    model = LPRNet(lpr_max_len, phase, class_num, dropout_rate)

    state_dict = torch.load(weight_path)
    model.load_state_dict(state_dict)

    return model

model = load_model("./weights/Final_LPRNet_model.pth")

<ipython-input-4-608e68d2df59>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path)


In [5]:
print("Weights of the first Convolutional Layer:")
print(model.backbone[0].weight.data.shape)

print("\nWeights of the first small_basic_block:")
first_basic_block = model.backbone[4].block[0]
print(first_basic_block.weight.data.shape)

Weights of the first Convolutional Layer:
torch.Size([64, 3, 3, 3])

Weights of the first small_basic_block:
torch.Size([32, 64, 1, 1])


In [6]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [ ]:
module = model.backbone
print(list(module.named_parameters()))

In [ ]:
print(model.state_dict().keys())

odict_keys(['backbone.0.weight', 'backbone.0.bias', 'backbone.1.weight', 'backbone.1.bias', 'backbone.1.running_mean', 'backbone.1.running_var', 'backbone.1.num_batches_tracked', 'backbone.4.block.0.weight', 'backbone.4.block.0.bias', 'backbone.4.block.2.weight', 'backbone.4.block.2.bias', 'backbone.4.block.4.weight', 'backbone.4.block.4.bias', 'backbone.4.block.6.weight', 'backbone.4.block.6.bias', 'backbone.5.weight', 'backbone.5.bias', 'backbone.5.running_mean', 'backbone.5.running_var', 'backbone.5.num_batches_tracked', 'backbone.8.block.0.weight', 'backbone.8.block.0.bias', 'backbone.8.block.2.weight', 'backbone.8.block.2.bias', 'backbone.8.block.4.weight', 'backbone.8.block.4.bias', 'backbone.8.block.6.weight', 'backbone.8.block.6.bias', 'backbone.9.weight', 'backbone.9.bias', 'backbone.9.running_mean', 'backbone.9.running_var', 'backbone.9.num_batches_tracked', 'backbone.11.block.0.weight', 'backbone.11.block.0.bias', 'backbone.11.block.2.weight', 'backbone.11.block.2.bias', 'ba

In [8]:
parameters_to_prune = []

for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        parameters_to_prune.append((module, 'weight'))

In [9]:
parameters_to_prune

[(Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1)), 'weight'),
 (Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)), 'weight'),
 (Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1)), 'weight'),
 (Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)), 'weight'),
 (Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1)), 'weight'),
 (Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)), 'weight'),
 (Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1)), 'weight'),
 (Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 256, kernel_size=(1, 4), stride=(1, 1)), 'weight'),
 (Co

In [10]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.9,
)

In [11]:
for module, _ in parameters_to_prune:
    prune.remove(module, 'weight')

torch.save(model.state_dict(), './weights/pruned_model_weights_trial2.pth')

In [12]:
parameters_to_prune_first_4 = []
parameters_to_prune_next_8 = []
parameters_to_prune_last_4 = []

conv2d_count = 0
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        if conv2d_count < 4:
            parameters_to_prune_first_4.append((module, 'weight'))
        elif conv2d_count < 12:
            parameters_to_prune_next_8.append((module, 'weight'))
        else:
            parameters_to_prune_last_4.append((module, 'weight'))

        conv2d_count += 1

prune.global_unstructured(
    parameters_to_prune_first_4,
    pruning_method=prune.L1Unstructured,
    amount=0.2,  # 20% pruning for first 4 layers
)

prune.global_unstructured(
    parameters_to_prune_next_8,
    pruning_method=prune.L1Unstructured,
    amount=0.9,  # 50% pruning for next 8 layers
)

prune.global_unstructured(
    parameters_to_prune_last_4,
    pruning_method=prune.L1Unstructured,
    amount=0.9,  # 80% pruning for last 4 layers
)

In [13]:
for module, _ in parameters_to_prune_first_4:
    prune.remove(module, 'weight')

for module, _ in parameters_to_prune_next_8:
    prune.remove(module, 'weight')

for module, _ in parameters_to_prune_last_4:
    prune.remove(module, 'weight')

torch.save(model.state_dict(), './weights/pruned_model_weights_trial.pth')

In [ ]:
parameters_to_prune_first_4

[(Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1)), 'weight'),
 (Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)), 'weight'),
 (Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1)), 'weight')]

In [ ]:
parameters_to_prune_next_8

[(Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)), 'weight'),
 (Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1)), 'weight'),
 (Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)), 'weight'),
 (Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1)), 'weight')]

In [ ]:
parameters_to_prune_last_4

[(Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)), 'weight'),
 (Conv2d(64, 256, kernel_size=(1, 4), stride=(1, 1)), 'weight'),
 (Conv2d(256, 68, kernel_size=(13, 1), stride=(1, 1)), 'weight'),
 (Conv2d(516, 68, kernel_size=(1, 1), stride=(1, 1)), 'weight')]

TVM

In [ ]:
import tvm
from tvm.relax.frontend.torch import from_exported_program
import torch
from torch.export import export

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed